# Massive Machine Learning Pipelines with scikit-learn

The objective is to build a scikit-learn pipeline that takes in raw data, transforms many different subsets of data, learns from it, tunes hyperparameters and is then able to make predictions. The final outcome will be a single sckit-learn estimator that does it all. This estimator will then be able to be used in the future to make predictions in a single step.

This tutorial includes information on how to:
* Categorize columns into distinct groups
* Apply independent transformations to each column grouping
* Create a single pipline that handles all the steps of transforming, modeling, validating, and predicting
* Build custom transformers to create new features
* Save our final model to disk to be used to make future predictions

## Focus on transformations
Although this builds a complete machine learning pipeline, most of it will focus on how to transform and prepare the data for the machine learning models.

## Assume fundamentals of scikit-learn
This tutorial assumes you are familiar with doing machine learning with scikit-learn. At a minimum, you need to know what a scikit-learn estimator is and how it behaves

## Prior Issues with transformations in scikit-learn
Up until the release of scikit-learn version 0.20 in September, 2018, there was no easy way to apply separate transformations to different subsets of the data. Additionally burdensome, was encoding of categorical features. For instance, building a single pipeline to handle input data that contained a mix of continuous and categorical variables was not trivial and a huge issue. Many divergent workflows were built to accommodate such transformations.

The addition of the `ColumnTransformer` and upgrade to the `OneHotEncoder` in version 0.20 alleviated these painful issues. Building an entire pipeline in scikit-learn was not pleasant before these additions. A previous post of mine details the exciting new workflow that became possible.

## Real-World Data
In this post, we will work with the [Ames, Iowa housing dataset][0] from a popular Kaggle competition. You are given a dataset with 79 features with the objective of learning a model to predict the sale price.

[0]: https://www.kaggle.com/c/house-prices-advanced-regression-techniques

## Begin by reading in data
Let's begin by reading our training data into a pandas DataFrame.

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 100)
housing = pd.read_csv('data/train.csv')
housing.head(3)

Output the number of rows and columns.

In [ ]:
housing.shape

### Remove the sale price
The target variable to predict is the sale price. We will remove it and assign to the variable `y` with the `pop` method, which modifies the DataFrame in place.

In [ ]:
# Running this cell twice will cause an error since the columns are no longer in the DataFrame
y = housing.pop('SalePrice').values
y[:5]

## Use the data dictionary to gain a deeper understanding of the problem

A very useful data dictionary is provided that gives descriptions on each column in the dataset. We will rely on it to understand the meaning of the columns and help with making more logical transformations.

Its contents may be printed out directly into the notebook for easy access. For now, look at the first column. It's composed of numeric values, but they are just codes for a type of house.

In [ ]:
print(open('data/data_description.txt').read())

## Classifying columns into groups
One of the most important tasks during this tutorial is going to be the classification of each column into a particular group. Each group will undergo its own set of transformations before being used as input in the machine learning model. A commonly taught approach is to classify each column as either **categorical** or **continuous**. Values in categorical columns are discrete and typically the total number of categories is known. Continuous columns are always numeric and not limited to a known set of values.

### Different transformations for each group
Categorical columns are processed differently than continous ones with each group needing its own set of transformations. For instance, we may want to fill in missing values with the most frequent for categorical, but the mean for continuous. Categorical columns are often one-hot encoded, while continuous columns are often standardized. As we will see, there are different groups of columns than just categorical or continuous and each column group will have its own set of transformations.

## A Mini Machine Learning Pipeline
Before we embark on our massive machine learning pipeline, we'll create a much simpler one that consists of a few columns that are either categorical or continuous. Creating this small pipeline will help understand the larger one.

### Select columns for each group
We'll begin by assigning a few categorical and continuous columns as lists. The columns here are chosen arbirtrarily as the focus is going to be on applying the transformations and building the pipeline.

In [ ]:
cat_cols = ['Neighborhood', 'LotShape', 'OverallQual', 'MasVnrType']
cont_cols = ['GrLivArea', 'GarageArea', 'LotFrontage']

### Build a pipeline for each column group
Each group will go through two transformations. For the categorical columns, the missing values will be filled with the most frequent and then one-hot encoded. For the continuous columns, the missing values will be filled with the mean and then standardized. A scikit-learn `Pipeline` can be used whenever there are two or more transformations that are needed to be applied in succession.

Before we biuld the pipeline, we will import each of the transformers and instantiate them. Notice that the `OneHotEncoder` is constructed with the `handle_unknown` parameter set to 'ignore'. This will help us when predicting new data that have categories not seen in the training set.

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

si_cat = SimpleImputer(strategy='most_frequent')
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

si_cont = SimpleImputer(strategy='mean')
ss = StandardScaler()

We can now import the scikit-learn `Pipeline` to create the actual pipelines for each column group. To instantiate a `Pipeline`, create a list of two-item tuples where each tuple consists of a **name** and the **transformer**. The name is an arbitrary string that may be used to reference the transformer at a later time. Below, we create two pipelines, `cat_pipe` and `cont_pipe`.

In [ ]:
from sklearn.pipeline import Pipeline

cat_steps = [
    ('si', si_cat),
    ('ohe', ohe)
]
cat_pipe = Pipeline(cat_steps)

cont_steps = [
    ('si', si_cont),
    ('ss', ss)
]
cont_pipe = Pipeline(cont_steps)

By default, scikit-learn will apply the transformations in a pipeline to all of the columns. But, we are only interested in passing the categorical columns through the categorical pipeline and the continuous columns through the continuous pipeline. The excellent `ColumnTransformer` allows us to do this. We instantiate it with a list of three-item tuples where each tuple consists of a **name** of the transformation, the **transformer**, and a list of **columns** to apply the transformation to. In our case, the transformer is a pipeline of individual transformers.

In [ ]:
from sklearn.compose import ColumnTransformer

transformers = [
    ('cat_cols', cat_pipe, cat_cols),
    ('cont_cols', cont_pipe, cont_cols)
]
ct = ColumnTransformer(transformers)

### Visualizing the ColumnTransformer
At this point, we have constructed the machinery to do the transformations. We haven't actually done any transformations, but we are ready to do so. The following shows how our data would flow at this point. Our raw data would be passed to the `ColumnTransformer` which will send the categorical column to the categorical pipeline and the continuous columns to the continuous pipelines. Each pipeline will apply two successive transformations to the data. After each pipeline has completed, the `ColumnTransformer` concatenates the data back together to form a singe transformed dataset.

![](images/simple_columntransformer.png)

### Passing the data through the `ColumnTransformer`

Let's pass our data through the `ColumnTransformer` to obtain our final transformed dataset. Only the columns that appear in either the `cat_cols` or `cont_cols` lists will be transformed. Any other columns will be dropped. Pass in the pandas DataFrame to the `fit_transform` method and you will be returned a numpy array of the transformed data.

In [ ]:
X_t = ct.fit_transform(housing)
type(X_t)

Let's see the shape of the new dataset.

In [ ]:
X_t.shape

We transformed 7 columns and were returned 46. This is entirely due to one-hot encoding. All the other transformers mapped each input column to exactly one output column.

### Create one more pipeline to do machine learning
Our data is now ready to be fed into a machine learning model. We could use the variable `X_t` that was created above, but instead, we will build another pipeline where the first step passes the data through the `ColumnTransformer` and the second passes the transformed data to the machine learning model. We'll use Ridge Regression for learning. Let's create this final two-step pipeline.

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge()

steps = [
    ('ct', ct),
    ('ridge', ridge)
]

final_pipe = Pipeline(steps)

### Visualizing the final pipeline

Our final pipeline is a bit complex. It consists of only two steps, but the ColumnTransformer in the first step contains two separate pipelines and each of those pipelines contains two transformations. The image below shows how the data is processed.

![](images/final_pipeline.png)

### Train the model
Let's use this pipeline to pass in train a Ridge regression model. We simply pass the original DataFrame to the `fit` method which will run all the transformations and train the model.

In [ ]:
final_pipe.fit(housing, y);

### Making predictions
After training, we can now make predictions on new dataets that have the same column names as the original. Let's read in the test dataset and assign the Id column to its own variable.

In [ ]:
housing_test = pd.read_csv('data/test.csv')
housing_test.head(3)

We now pass in the test set to the pipeline's `predict` method to get the predictions.

In [ ]:
y_pred = final_pipe.predict(housing_test)
y_pred[:5]

## Saving the model to disk for future use
We can preserve our trained pipeline exactly as it is by saving it to disk with help from the `joblib` library `dump` function. See [joblib documentation][0] for more information. Pass it the pipeline and a name for the new file.

[0]: https://joblib.readthedocs.io/en/latest/persistence.html#persistence

In [ ]:
import joblib
joblib.dump(final_pipe, 'models/minipipeline_ridge.joblib')

## Retrieve the saved model
We can retrieve the model from disk with the `load` function. It has preserved every step of the pipeline. We test that the results are the same by testing that the predictions are the same as the original.

In [ ]:
final_pipe_new = joblib.load('models/minipipeline_ridge.joblib')
y_pred_new = final_pipe_new.predict(housing_test)
(y_pred == y_pred_new).all()

## Submit to Kaggle

We now have a prediction for each test observation and can submit them to Kaggle to get scored and ranked against the other competitors.

### Create a csv of Id and SalePrice
We need to submit a csv file of the row Id and our predicted sale price. We do so with the DataFrame constructor to create a two-column DataFrame.

In [ ]:
sub01 = pd.DataFrame({'Id': housing_test['Id'], 'SalePrice': y_pred})
sub01.head(3)

### File naming and directory structure

From here we can export our DataFrame as a csv. I strongly recommend creating a submissions folder within the data folder and within that folder a new folder for each date that you track submissions. Within that folder is where the submission files will be saved.

In [ ]:
sub01.to_csv('data/submissions/20190710/sub01.csv', index=False)

Our directory structure for the data takes the following shape:

![](images/dir.png)

### Make a submission to Kaggle from python
Kaggle has kindly provided a [python library][0] (`pip install kaggle`) to make submissions programmatically. You'll need to read the documentation to learn how to authenticate your account.

[0]: https://github.com/Kaggle/kaggle-api

In [ ]:
import kaggle

We submit our csv by passing the submission function our file location, a message, and the competition name. It's important to give a good descriptive message so that you can remember how that particular submission was created.

In [ ]:
file = 'data/submissions/20190710/sub01.csv'
message = '''
One hot encoded four categorical columns and standardized three continuous
columns. Modeled with ridge regression with alpha=1
'''
competition = 'house-prices-advanced-regression-techniques'
# kaggle.api.competition_submit(file, message, competition)

### Retrieve score
We can retrieve a list of all of our prior scores with the following function call. The most recent submission will be the first item in the list which we print to the screen.

In [ ]:
all_submissions = kaggle.api.competitions_submissions_list(competition)
all_submissions[0]

The competition uses root mean squared error of the logged housing prices. Our score is .18. Unfortunately there is no easy way to get our place on the leaderboard, but you can just navigate [directly to the Kaggle leaderboard][0] instead.

[0]: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/leaderboard

## Pipeline vs ColumnTransformer
It may not be crystal clear what the difference is between the `Pipeline` and `ColumnTransformer`. The simplest distinction is to think of the `Pipeline` as moving all of the data **vertically** in succession from one transformer to the next. The `ColumnTransformer` splits data **horizontally** into multiple subsets. Each of these subsets gets applied a transformation that is independent of all the other subsets of data. All the transformed subsets are then concatenated together to form a single dataset.

In a `Pipeline`, all columns will be passed into each estimator with the result being passed to the next estimator. A `Pipeline` allows for the last step to be a machine learning model whereas the `ColumnTransformer` only allows transformers.

### All steps in one cell
All the above steps for the our mini machine learning pipeline were written in different cells, which probably makes it harder to see the full structure of the program as it would outside of a Jupyter Notebook/Tutorial

In [ ]:
si_cat = SimpleImputer(strategy='most_frequent')
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

si_cont = SimpleImputer(strategy='mean')
ss = StandardScaler()

cat_cols = ['Neighborhood', 'LotShape', 'OverallQual', 'MasVnrType']
cont_cols = ['GrLivArea', 'GarageArea', 'LotFrontage']

cat_steps = [('si', si_cat), ('ohe', ohe)]
cat_pipe = Pipeline(cat_steps)

cont_steps = [('si', si_cont), ('ss', ss)]
cont_pipe = Pipeline(cont_steps)

transformers = [('cat_cols', cat_pipe, cat_cols), ('cont_cols', cont_pipe, cont_cols)]
ct = ColumnTransformer(transformers)

steps = [('ct', ct), ('ridge', ridge)]
final_pipe = Pipeline(steps)
final_pipe.fit(housing, y)
y_pred = final_pipe.predict(housing_test)
sub01 = pd.DataFrame({'Id': housing_test['Id'], 'SalePrice': y_pred})
sub01.to_csv('data/submissions/20190710/sub01.csv', index=False)

## Massive Machine Learning Pipeline
We will move on to building the massive machine learning pipeline. The overall architecture will look similar to the mini-pipeline from above with the major difference being the number of distinct column groups. Each of the column groupings we create will have its own pipeline.

## Create column groupings
For this massive pipeline, we will use all of the columns. Typically, it's a good idea to do exploratory data analysis first to manually inspect the columns and possibly select a subset to model on. We will forgo this step and go straight into pipeline construction. Further along in the tutorial we will do some data inspection and feature engineering.

Below, we separate the data into five separate column groupings. The data type of each column is either numeric or string. Both numeric and string columns can be categorical, but only numeric data can be continuous. Categorical data is further subdivided into nominal (no natural ordering) or ordinal (has a natural ordering - basement quality for example). The data dictionary was used to help classify each column correctly.

In [ ]:
str_nomial = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 
              'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
              'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
              'Heating', 'CentralAir', 'Electrical', 'GarageType', 'GarageFinish', 'PavedDrive',
              'MiscFeature', 'SaleType', 'SaleCondition']
str_ordinal = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
               'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 'GarageQual', 'GarageCond',
               'PoolQC', 'Fence', 'FireplaceQu']

numeric_nominal = ['MSSubClass', 'MoSold', 'YrSold', 'YearBuilt', 'YearRemodAdd', 'GarageYrBlt']
numeric_ordinal = ['OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
                   'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', ]
numeric_cont = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                'TotalBsmtSF','1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 
                'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
                'MiscVal']

### Nominal vs Ordinal
Most of the time it is easy to determine whether a categorical column is nominal or ordinal. The neighborhood column, for instance has no natural ordering and would therefore be classified as nominal. 

But with a column like LotShape, it isn't so clear. The values are Reg (Regular), IR1 (Slightly Irregular), IR2 (Moderately Irregular), and IR3 (Irregular). The values appear to have an order with Reg being the 'best' and IR3 the 'worst'. But, without being an expert in this field, it's probably safer to assume less and treat it as a nominal.

### One-Hot encoding all categorical columns
While scikit-learn does provide an `OrdinalEncoder` to encode ordinal features, we will choose opt to one-hot encode them instead. This is done for a couple reasons. First, ordinal encoding places a stricter assumption on the data, that each category is worth precisely one more than the prior one. Second, we would need to provide an ordered list of categories for each column and that would be quite tedious. So, partly out of laziness, we will one-hot encode all categorical columns regardless if they are nominal or ordinal.

### Create pipeline steps for each column group

We will now create a pipeline for each subset of columns. We will fill missing values with the most frequent for all of the categorical columns and with the median for the continuous.

In [ ]:
str_nominal_steps = [
    ('si', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
]
str_orinal_steps = [
    ('si', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
]
numeric_nominal_steps = [
    ('si', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
]
numeric_ordinal_steps = [
    ('si', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
]
numeric_cont_steps = [
    ('si', SimpleImputer(strategy='median')),
    ('ss', StandardScaler())
]

### Create pipeline for each column group
We instantiate each pipeline with the steps (list of two-item tuples) from above. Notice, that all pipelines are identical except for the continuos one. We could have used just two pipelines to transform the data like we did in our first attempt. In upcoming examples, we will see different sets of transformations applied to each pipeline. Regardless, it is still useful to see how different columns many be categorized.

In [ ]:
str_nominal_pipe = Pipeline(str_nominal_steps)
str_ordinal_pipe = Pipeline(str_orinal_steps)
numeric_nominal_pipe = Pipeline(numeric_nominal_steps)
numeric_ordinal_pipe = Pipeline(numeric_ordinal_steps)
numeric_cont_pipe = Pipeline(numeric_cont_steps)

### Create ColumnTransformer
We instantiate our `ColumnTransformer` with a list of three-item tuples.

In [ ]:
transformers = [
    ('str_nominal_pipe', str_nominal_pipe, str_nomial),
    ('str_ordinal_pipe', str_ordinal_pipe, str_ordinal),
    ('numeric_nominal_pipe', numeric_nominal_pipe, numeric_nominal),
    ('numeric_ordinal_pipe', numeric_ordinal_pipe, numeric_ordinal),
    ('numeric_cont_pipe', numeric_cont_pipe, numeric_cont)
]
ct = ColumnTransformer(transformers)

### Create one last pipeline to add machine learning
A final pipeline is created to connect the transformed data to the Ridge regression model.

In [ ]:
final_steps = [
    ('ct', ct),
    ('ridge', Ridge())
]
final_pipe2 = Pipeline(final_steps)

### Fit and predict
We can now pass our data into the final pipeline to transform and ultimately train our ridge regression model. We follow this up by predicting the sale price for the test set and saving a new submission file.

In [ ]:
final_pipe2.fit(housing, y)
y_pred = final_pipe2.predict(housing_test)

### Automate submission to kaggle
We can define a function to create the csv, submit the prediction, and return the score.

In [ ]:
def submit_kaggle(model, X_test, file, message):
    y_pred = model.predict(X_test)
    sub = pd.DataFrame({'Id': X_test['Id'], 'SalePrice': y_pred})
    sub.to_csv(file, index=False)
    kaggle.api.competition_submit(file, message, competition)

In [ ]:
file = 'data/submissions/20190710/sub02.csv'
message = '''
One hot encoded all categorical columns and standardized all continuous
columns. Modeled with ridge regression with alpha=1
'''
# submit_kaggle(final_pipe2, housing, file, message)

### Cross Validation

It's pretty disappointing to see all that hard work separating columns into groups doing absolutely nothing for our score. Since Kaggle only allows 10 submissions per day, it would have been nice to know that our model wouldn't show an improvement before sending our submission.

We can get a fairly good idea of how our model is going to perform by doing cross validation. Below, we perform K-Fold cross validation with 5 folds. By default, the r-squared value will be returned as the score. Instead, we can return the negative mean of the squared log error, which is nearly the same error calculation that Kaggle uses.

In most all other machine learning contexts, the root mean squared error is reported as a positive number. scikit-learn reports it as a negative number. This is strange, but is done because it considers a greater score to be better and ranks models based on their score when grid searching.

In [ ]:
# warning - this might not work for other random states
from sklearn.model_selection import cross_val_score, KFold
kf = KFold(n_splits=5, shuffle=True, random_state=123)
scores = cross_val_score(final_pipe2, housing, y, 
                         cv=kf, scoring='neg_mean_squared_log_error')
scores

To report the same metric as Kaggle, we will negate the scores, take the square root, and then average all of the scores. Because we are shuffling the data during cross validation, you'll get a different score each time, but it should be around .18, which indicates that there would be no improvement over the first model.

In [ ]:
import numpy as np
np.mean(np.sqrt(-scores))

### Transforming the target variable
Instead of using the actual sale price as the target variable, we can instead use the log of it. This is common practice when modeling data that with a large range of values. Kaggle is already scoring based on the log of the sale price so it makes sense to use it directly. 

scikit-learn provides the `TransformedTargetRegressor` meta-estimator to automate the transforming of any continuous target variable. We pass it our pipeline, the transformation function applied to the target and a function to invert the transformation. The model will be trained against the transformed target variable (the log of the sale price in our example). When predicting, the inverse function will be used to transform the target back to the original units.

Let's use this new estimator to train our model and make predictions.

In [ ]:
from sklearn.compose import TransformedTargetRegressor
final_pipe2_ttr = TransformedTargetRegressor(final_pipe2, func=np.log, inverse_func=np.exp)
final_pipe2_ttr.fit(housing, y)
final_pipe2_ttr.predict(housing_test)

Now that we verified the model can make predictions, let's use cross validation to get a estimate a score.

In [ ]:
scores = cross_val_score(final_pipe2_ttr, housing, y, 
                         cv=kf, scoring='neg_mean_squared_log_error')
np.mean(np.sqrt(-scores))

The score should have dropped to around .16 which gives us confidence that we will have some improvement over the original. Let's make our submission to Kaggle.

In [ ]:
file = 'data/submissions/20190710/sub03.csv'
message = 'Same as 2, but trainined on log of y.'
# submit_kaggle(final_pipe2_ttr, housing_test, file, message)

Our score from kaggle is .14 which is better than what was predicted by cross validation and far better than our original

In [ ]:
all_submissions = kaggle.api.competitions_submissions_list(competition)
all_submissions[0]

### Overfitting
Our model with all of the columns produced a similar score as the first trained on just a fraction of the columns when modeled against the actual sales price. This is likely due to the extreme flexibility that we have given the model. We used all 79 variables and one-hot encoded most of them, which created an enormous number of features. Let's see exactly how many features were created by running our data just through the `ColumnTransformer`.

In [ ]:
ct.transform(housing).shape

We built our model on input data with over 600 features or nearly half the total number of observations. If we are to continue to use this high number of features, we'll need to choose a different type of model or apply constraints to this one.

### Regularization
Ridge regression constrains the parameters of the model by applying a penalty term (`alpha`) preventing the model from learning too much from the training data. We use the `GridSearchCV` meta-estimator to find the best possible `alpha` from 100 given between .01 and 1000. By default, `alpha` is set to 1.

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'regressor__ridge__alpha': np.logspace(-2, 3, 100)}
gs = GridSearchCV(final_pipe2_ttr, param_grid, cv=kf, scoring='neg_mean_squared_log_error')
gs.fit(housing, y);

Let's get the best value for alpha.

In [ ]:
gs.best_params_

And the score associated with that alpha which is still right around .15.

In [ ]:
np.sqrt(-gs.best_score_)

We can assign the best model trained to its own variable and then get predictions.

In [ ]:
final_pipe2_ttr_best = gs.best_estimator_
y_pred = final_pipe2_ttr_best.predict(housing_test)
y_pred

Even with the best value for alpha, our ridge regression produced just about the same score. We won't bother submitting it to Kaggle. Let's try reducing the total number of features instead.

## Feature Engineering

All of the above represents a fairly 'vanilla' massive machine learning pipeline. Summarizing, we have done the following:
* Classified each column as either string or numeric and as either nominal, ordinal, or continuous
* Built a pipeline for each of these 5 column groupings by filling in missing values and either one-hot encoding or standardizing
* Used cross validation to estimate the root mean squared log error on the test set
* Modeled on the log of the sale price
* Used grid search to find the optimal value for the penalty term for ridge regression

We have essentially reached the limit of our learning without doing too much thinking. True, we did not choose a more complex type of model like a Random Forest or Gradient Boosted Tree. If we desire a better model, we will need to think about how to engineer better features from our data. 

### Replacing low-frequency categorical values

Let's look at individual columns for particular values that appear very few times (sometimes referred to as outliers).  Categorical values that appear infrequently are candidates to be reclassified as a another similar category or to be grouped together with other infrequent categories into an 'other' category.

### Why recclassify low-frequency categoricals?
A primary goal of machine learning is to build a model that generalizes well to future, unseen data. If our model is built with too many low-frequency categorical values, it may overfit to those particular categories. As a concrete example, imagine that there are just 2 houses in our training data from a particular neighborhood and both of these houses, by chance, just happen to be very poor quality houses and are not representative of the entire neighborhood. Our model might unfairly give too much negative weight to that neighborhood and then make poor predictions in the future.

Of course, this isn't always the case and a single unique category can actually give useful information. Perhaps there is a single house that has a solid-gold toilet that massively increases the value of the house. 

But, in general, I like to experiment with consolidating low-frequency categories so that the model can generalize better.

### Finding low-frequency categoricals
The `value_counts` Series method find the number of times each category appears. Let's see an example

In [ ]:
housing['LotConfig'].value_counts()

In this example, the `LotConfig` feature has 5 unique values but 'FR3' only appears 4 times. By looking at the data dictionary, we can see its description is similar to that of FR2, so we can consider replacing it.

### An automated way to find low-frequency categoricals
We can loop through each column and run the `value_counts` method on it if it is a string column ('object' in pandas). There are several columns in this dataset that are numeric, but represent discrete categories such as we saw with the first column 'MSSubClass'. We can also add a condition to run `value_counts` if the number of unique values is below a certain threshold. Below, only the columns that have a one category that appears 5 or fewer times is printed to the screen.

In [ ]:
for col in housing.columns:
    if housing[col].dtype == 'object' or housing[col].nunique() < 30:
        vc = housing[col].value_counts(dropna=False)
        if vc.min() <= 5:
            print(f'\nColumn {col}')
            print(vc)

In [ ]:
print(open('data/data_description.txt').read())

Remove completely
* Utilities, Condition2, RoofMatl, Heating, LowQualFinSF, PoolArea, MSSubClass
* MSSubClass is just about a 1-1 correlation with HouseStyle

### Replacing with `replace`

We can use the pandas DataFrame `replace` method helps us replace values within particular columns. We do this by creating a dictionary mapping the column name to another dictionary that maps the value to be replaced with its replacement value.

In [ ]:
replace_dict = \
{
    'LotConfig': {'FR3': 'FR', 'FR2': 'FR'},
    'Condition1': {'PosA': 'Pos', 
                   'PosN': 'Pos', 
                   'RRAe': 'RR', 
                   'RRNe': 'RR', 
                   'RRAn': 'RR', 
                   'RRNn': 'RR'},
    'OverallQual': {1: 2},
    'OverallCond': {1: 2},
    'Exterior1st': {'BrkComm': 'OTHER', 
                    'Stone': 'OTHER', 
                    'AsphShn': 'OTHER',
                    'CBlock': 'OTHER',
                    'ImStucc': 'OTHER',
                    'Other': 'OTHER',
                    'Brk Cmn': 'OTHER'},
    'Exterior2nd': {'BrkComm': 'OTHER', 
                    'Stone': 'OTHER', 
                    'AsphShn': 'OTHER',
                    'CBlock': 'OTHER',
                    'ImStucc': 'OTHER',
                    'Other': 'OTHER',
                    'Brk Cmn': 'OTHER'},
    'ExterCond':{'Po': 'Fa', 
                 'Ex': 'Gd'},
    'Foundation': {'Stone': 'OTHER',
                   'Wood': 'OTHER'},
    'BsmtCond': {'Po': 'Fa'},
    'Functional': {'Sev': 'Maj',
                   'Maj1': 'Maj',
                   'Maj2': 'Maj',
                   'Min1': 'Min',
                   'Min2': 'Min'},
    'HeatingQC': {'Po': 'Fa'},
    'GarageQual': {'Ex': 'Gd', 'Po': 'Fa'},
    'GarageCond': {'Ex': 'Gd', 'Po': 'Fa'},
    'MoSold': {12: 'Winter', 1: 'Winter', 2: 'Winter',
                3: 'Spring', 4: 'Spring',5: 'Spring',
                6: 'Summer', 7: 'Summer',8: 'Summer',
                9: 'Fall', 10: 'Fall', 11: 'Fall'}
}

keep_dict = {
    'RoofStyle': ['Gable', 'Hip'],
    'SaleType': ['WD', 'New', 'COD'],
    'Heating':  ['GasA', 'GasW']
}

clip_dict = {
    'BsmtFullBath': {'upper': 2},
    'BedroomAbvGr': {'lower': 1, 'upper': 5},
    'TotRmsAbvGrd': {'lower': 3, 'upper': 11},
    'Fireplaces': {'upper': 2},
    'GarageCars': {'upper': 3}
}

binarizer_dict = {
    'LotShape': ['Reg', True],
    'LandSlope': ['Gtl', False],
    'BsmtHalfBath': [0, True],
    '3SsnPorch': [0, True],
    'WoodDeckSF': [0, True],
    'EnclosedPorch': [0, True],
    'OpenPorchSF': [0, True],
    'ScreenPorch': [0, True],
    'PoolQC': [lambda x: x.notna(), False],
    'MiscFeature': ['Shed', False],
    'Electrical': ['SBrkr', False]
}

percentile_dict = {
    'LowQualFinSF': 'GrLivArea'
    '2ndFlrSF': 'GrLivArea',
    'BsmtFinSF1': 'TotalBsmtSF', 
    'BsmtFinSF2': 'TotalBsmtSF', 
    'BsmtUnfSF': 'TotalBsmtSF'
}

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class Replacer(BaseEstimator, TransformerMixin):
    
    def __init__(self, replace_dict=None, keep_dict=None):
        self.replace_dict = replace_dict
        self.keep_dict = keep_dict
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        if self.replace_dict:
            X = X.replace(self.replace_dict)
        else:
            X = X.copy()
        if self.keep_dict:
            for col, keep_vals in self.keep_dict.items():
                keep = X[col].isin(keep_vals)
                X[col] = X[col].where(keep, 'OTHER')
        return X
    
class Clipper(BaseEstimator, TransformerMixin):
    
    def __init__(self, clip_dict):
        self.clip_dict = clip_dict
        
    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        X = X.copy()
        for col, clip_kwargs in self.clip_dict.items():
            X[col] = X[col].clip(**clip_kwargs)
        return X
    
class MyBinarizer(BaseEstimator, TransformerMixin):
    
    def __init__(self, binarize_dict):
        self.binarize_dict = binarize_dict
        
    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        X = X.copy()
        for col, (item, reverse) in self.binarize_dict.items():
            if callable(item):
                X[col] = item(X[col])
            else:
                X[col] = X[col] == item
            if reverse:
                X[col] = 1 - X[col]
        return X
    
class Percentiler(BaseEstimator, TransformerMixin):
    
    def __init__(self, percentile_dict=None):
        self.percentile_dict = percentile_dict
        
    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        X = X.copy()
        for col, total_col in self.percentile_dict.items():
            X[col] = X[col] / X[total_col]
        return X

In [ ]:
not_used = ['Utilities', 'Condition2', 'RoofMatl', 'Heating', 
            'PoolArea', 'MSSubClass', '1stFlrSF']

str_nomial = ['MSZoning', 'Street', 'Alley', 'LandContour', 'LotConfig', 
              'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle',
              'RoofStyle', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
              'CentralAir', 'GarageType', 'GarageFinish', 'PavedDrive',
              'SaleType', 'SaleCondition', 'MoSold']
str_ordinal = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
               'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 
               'Functional', 'GarageQual', 'GarageCond', 'Fence', 'FireplaceQu']

numeric_nominal = ['YrSold']
numeric_ordinal = ['OverallQual', 'OverallCond', 'FullBath', 'HalfBath',
                   'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars']
numeric_cont = ['LotFrontage', 'LotArea', 'MasVnrArea', 'TotalBsmtSF', 
                'GrLivArea', 'GarageArea']
numeric_years = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']
numeric_perc = ['LowQualFinSF', '2ndFlrSF', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF']

all_binarized = ['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 
                 'BsmtHalfBath', 'LotShape', 'LandSlope', 'PoolQC', 'MiscFeature', 'Electrical']

from sklearn.preprocessing import KBinsDiscretizer

numeric_years_steps = [
    ('si', SimpleImputer(strategy='median')),
    ('kbd', KBinsDiscretizer(n_bins=5, encode='onehot-dense', strategy='kmeans'))
]
numeric_perc_steps = [
    ('si', SimpleImputer(strategy='constant', fill_value=0))
]
all_binarized_steps = [
    ('si', SimpleImputer(strategy='constant', fill_value=0))
]
numeric_years_pipe = Pipeline(numeric_years_steps)
numeric_perc_pipe = Pipeline(numeric_perc_steps)
all_binarized_pipe = Pipeline(all_binarized_steps)

transformers = [
    ('str_nominal_pipe', str_nominal_pipe, str_nomial),
    ('str_ordinal_pipe', str_ordinal_pipe, str_ordinal),
    ('numeric_nominal_pipe', numeric_nominal_pipe, numeric_nominal),
    ('numeric_ordinal_pipe', numeric_ordinal_pipe, numeric_ordinal),
    ('numeric_cont_pipe', numeric_cont_pipe, numeric_cont),
    ('numeric_years_pipe', numeric_years_pipe, numeric_years),
    ('numeric_perc_pipe', numeric_perc_pipe, numeric_perc),
    ('all_binarized_pipe', all_binarized_pipe, all_binarized)
]
ct = ColumnTransformer(transformers)

In [ ]:
mb = MyBinarizer(binarizer_dict)
clipper = Clipper(clip_dict)
replacer = Replacer(replace_dict, keep_dict)
percentiler = Percentiler(percentile_dict)
rp = Pipeline([('mb', mb), 
               ('clipper', clipper), 
               ('replacer', replacer),
               ('percentiler', percentiler),
               ('ct', ct), 
               ('ridge', ridge)])
rp_ttr = TransformedTargetRegressor(rp, func=np.log, inverse_func=np.exp)

In [ ]:
param_grid = {'regressor__ridge__alpha': np.logspace(-1, 3, 10)}
gs = GridSearchCV(rp_ttr, param_grid, cv=kf, scoring='neg_mean_squared_log_error')
gs.fit(housing, y);
rp_ttr_best = gs.best_estimator_

In [ ]:
gs.best_params_

In [ ]:
np.sqrt(-gs.best_score_)

In [ ]:
p = Pipeline([('mb', mb), ('clipper', clipper), ('replacer', replacer), 
              ('percentiler', percentiler), ('ct', ct)])
p.fit_transform(housing).shape

In [ ]:
file = 'data/submissions/20190710/sub04.csv'
message = 'binarize, clip, replace, years binned, percentiles: ridge(alph=6)'
# submit_kaggle(rp_ttr_best, housing_test, file, message)

In [ ]:
all_submissions = kaggle.api.competitions_submissions_list(competition)
all_submissions[0]

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfr = RandomForestRegressor()

In [ ]:
rp = Pipeline([('mb', mb), 
               ('clipper', clipper), 
               ('replacer', replacer),
               ('percentiler', percentiler),
               ('ct', ct), 
               ('rfr', rfr)])
rp_ttr_rfr = TransformedTargetRegressor(rp, func=np.log, inverse_func=np.exp)

In [ ]:
cross_val_score(est, X, y, cv=kf, )